## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json
import numpy as np

# Import API key
from configcopy import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv", index_col=0)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
City_ID,,,,,,,,,
0,Touros,BR,-5.1989,-35.4608,82.11,70,8,14.74,clear sky
1,Kodiak,US,57.7900,-152.4072,49.69,71,15,1.21,few clouds
2,Hermanus,ZA,-34.4187,19.2345,56.89,84,17,10.27,few clouds
3,Anzhero-Sudzhensk,RU,56.0810,86.0285,46.06,93,100,13.51,light rain
4,College,US,64.8569,-147.8028,62.58,58,70,1.01,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
City_ID,,,,,,,,,
4,College,US,64.8569,-147.8028,62.58,58,70,1.01,broken clouds
6,Avarua,CK,-21.2078,-159.7750,77.05,73,75,10.36,light rain
7,Rikitea,PF,-23.1203,-134.9692,73.94,78,56,12.37,broken clouds
8,Nikolskoye,RU,59.7035,30.7861,66.09,69,7,2.77,clear sky
11,Saint-Philippe,RE,-21.3585,55.7679,74.75,83,47,12.26,scattered clouds
13,Thyboron,DK,56.6985,8.2124,67.08,86,12,9.19,few clouds
14,Vaini,TO,-21.2000,-175.2000,73.72,69,47,18.21,scattered clouds
17,Batagay-Alyta,RU,67.8006,130.4114,60.76,50,100,9.17,overcast clouds
30,Padang,ID,-0.9492,100.3543,75.90,92,99,3.06,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   286
Country                283
Lat                    286
Lng                    286
Max Temp               286
Humidity               286
Cloudiness             286
Wind Speed             286
Weather Description    286
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City                   283
Country                283
Lat                    283
Lng                    283
Max Temp               283
Humidity               283
Cloudiness             283
Wind Speed             283
Weather Description    283
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
City_ID,,,,,,,
4,College,US,62.58,broken clouds,64.8569,-147.8028,
6,Avarua,CK,77.05,light rain,-21.2078,-159.7750,
7,Rikitea,PF,73.94,broken clouds,-23.1203,-134.9692,
8,Nikolskoye,RU,66.09,clear sky,59.7035,30.7861,
11,Saint-Philippe,RE,74.75,scattered clouds,-21.3585,55.7679,
13,Thyboron,DK,67.08,few clouds,56.6985,8.2124,
14,Vaini,TO,73.72,scattered clouds,-21.2000,-175.2000,
17,Batagay-Alyta,RU,60.76,overcast clouds,67.8006,130.4114,
30,Padang,ID,75.90,overcast clouds,-0.9492,100.3543,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    try:
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.

#Note: Because the hotel names were white space and not NaN, I had to enter
# "NaN" for the empty values. Then I dropped the NaN values using Dropna()
#Refernced Article for help: https://www.geeksforgeeks.org/working-with-missing-data-in-pandas/
hotel_NaN_df = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = hotel_NaN_df.dropna()
clean_hotel_df.head(20)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
City_ID,,,,,,,
4,College,US,62.58,broken clouds,64.8569,-147.8028,Pike's Waterfront Lodge
6,Avarua,CK,77.05,light rain,-21.2078,-159.7750,Paradise Inn
7,Rikitea,PF,73.94,broken clouds,-23.1203,-134.9692,Pension Maro'i
8,Nikolskoye,RU,66.09,clear sky,59.7035,30.7861,Tourist House - Sablino
11,Saint-Philippe,RE,74.75,scattered clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
13,Thyboron,DK,67.08,few clouds,56.6985,8.2124,Thyborøn Hotel
14,Vaini,TO,73.72,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
30,Padang,ID,75.90,overcast clouds,-0.9492,100.3543,Mercure Padang
32,Matara,LK,77.79,broken clouds,5.9485,80.5353,Amaloh Boutique Resort


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file)

In [13]:
# 9. Using the template add city name, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 

locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))